In [1]:
import torch

from cmprs.sparsification import RandomP, RandomK

D:\python3_venvs\lab\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
P = 0.5
d = 1_000
K = int(d * P)

In [3]:
rp = RandomP(p=P, seed=42)
rk = RandomK(k=K, seed=42)

In [4]:
x = torch.arange(1, d + 1).to(torch.float32)

In [5]:
N = 1000  # number of senders / clients
T = 10  # number of trials

In [10]:
def distributed_mean(transform):
    avg = torch.zeros_like(x)
    for _ in range(N):
        avg += transform.roundtrip(x).rx
    avg /= N
    return avg

In [11]:
def dme_mse(transform):
    avg = 0.
    for _ in range(T):
        avg += torch.sum((distributed_mean(transform) - x) ** 2)
    return avg / T

In [12]:
print(f'Distributed mean estimation {N=} {T=} {d=}')

Distributed mean estimation N=1000 T=10 d=1000


In [13]:
print(f'Random K({K=}) NMSE={dme_mse(rk) / torch.sum(x ** 2)}')

Random K(K=500) NMSE=0.0009921388700604439


In [14]:
print(f'Random P({P=}) NMSE={dme_mse(rp) / torch.sum(x ** 2)}')

Random P(P=0.5) NMSE=0.0009975723223760724
